In [5]:
from pathlib import *
import numpy as np
import pandas as pd

In [6]:
def get_subids(behavepath):
    subfiles="*study"
    sublist = []
    for filepathobj in behavepath.glob(subfiles):
        fname=filepathobj.name
        sub = fname[:3]
        sublist.append(sub)
    sublist.sort()
    return sublist

def set_behavior_path(sub, behavestring, extra):
    behaveobj=[behavestring+sub+extra]
    behavefilepath=Path(behaveobj[0])
    behavefilepath.exists()
    return behavefilepath

def read_study_file(filepath):
    """read in studyarray, turn into DataFrame and delete extra columns"""
    colnames=['obj1','obj2','obj3','cuecond','loc1','x1','y1','loc2','x2','y2','loc3','x3','y3',
     'dom_loc_rt','dom_loc_resp','cond','block','obj_type', 'dom_obj_id', 'dom_loc_actual', 'dom_choice_rt']
    studyarray=pd.read_table(filepath,header=None,names=colnames)
    tmpmask=~studyarray.columns.str.contains('tmp')
    studyarray=studyarray[studyarray.columns[tmpmask]]
    trialnum = np.arange(1,len(studyarray)+1)
    studyarray['studytrial'] = trialnum
    return studyarray

def read_test_file(filepath):
    """read in testarray, turn into DataFrame and delete extra columns"""
    colnames=['obj1','obj2','obj3','cuecond','loc1','x1','y1','loc2','x2','y2','loc3','x3','y3',
     'dom_loc_rt','dom_loc_resp','cond','block','obj_type', 'dom_obj_id', 'dom_loc_actual', 'dom_choice_rt',
             'tmp', 'tmp', 'test_obj_id', 'test_loc_id', 'tmp', 'tmp', 'tmp', 'tmp', 'test_resp', 'test_obj_rt',
             'test_loc_id', 'test_loc_rt', 'conf', 'tmp']
    testarray=pd.read_table(filepath,header=None,names=colnames)
    tmpmask=~testarray.columns.str.contains('tmp')
    testarray=testarray[testarray.columns[tmpmask]]
    trialnum = np.arange(1,len(testarray)+1)
    testarray['testtrial'] = trialnum
    testarray['recog_accuracy'] = testarray['test_resp'] == 1
    testarray['recog_loc_accuracy'] = (testarray['test_loc_id'] == testarray['loc1']) | (testarray['test_loc_id'] == testarray['loc2']) | (testarray['test_loc_id'] == testarray['loc3'])
    testarray['dom_accuracy'] = testarray['dom_loc_resp'] == testarray['dom_loc_actual']
    return testarray

# merge study & test
def merge_study_test(studyarray, testarray):
    # merge study & test
    testarray['obj1'] = testarray['obj1'] + (1000*testarray['obj_type'])
    testarray = testarray.set_index('obj1')
    studyarray['obj1'] = studyarray['obj1'] + (1000*studyarray['obj_type'])
    studyarray.sort_values('obj1')
    studyarray = studyarray.set_index('obj1')
    behavearray = studyarray.copy()

    for ind, ldf in testarray.iterrows():
        for col in ['dom_accuracy', 'recog_accuracy', 'recog_loc_accuracy', 'testtrial']:
            behavearray.loc[ind, col] = ldf[col]
    behavearray.reset_index(inplace=True)
    return behavearray

def adjust_pres_coords(array,x,y,xmax,ymax):
    """adjustment for behavioral coords to match
    eye coords for presentation version of exp"""
    newarray=pd.DataFrame()
    newarray[x]=array[x]+xmax
    newarray[y]=(array[y]-ymax)*-1
    return newarray

def apply_adjust_pres_coords(behavearray, sub):
    """applies adjust_pres_coords to all
    coords in behave array"""
           
    xs = [f'x{loc}' for loc in range(1,4)]
    ys = [f'y{loc}' for loc in range(1,4)]
    newlocs = pd.DataFrame()
    for loc in zip(xs, ys):
        x = loc[0]
        y = loc[1]
        newloc=adjust_pres_coords(behavearray,x,y, xmax, ymax)
        newlocs[x] = newloc[x]
        newlocs[y] = newloc[y]

    cols=newlocs.columns.tolist()
    for loc in cols:
        behavearray[loc]=newlocs[loc]
    return behavearray

In [7]:
def parse_eye_filename(pathobject):
    fname=pathobject.name
    parts=fname.split(".")[0]
    subject=parts[:3]
    block=parts[3:4]
    subdict={"subject":subject, "block":block, "fname":fname}
    return subdict

def get_eye_files(subids,eyepath):
    """ returns master dataframe including eye file name, block, phase, subid
    input list of subject strings, Path object pointing to eye files
    """
    substrings=[s+"*.asc" for s in subids]
    subinfo=[]
    for s in substrings:
        for filepathobj in eyepath.glob(s):
            subdict=parse_eye_filename(filepathobj)
            subinfo.append(subdict)

    masterdf=pd.DataFrame(subinfo).sort_values(by=["subject","block"])
#     print(masterdf.head())
    masterdf=masterdf[["subject","block","fname"]]
    masterdf = masterdf[~masterdf['fname'].str.contains('i')]
    masterdf.index=range(len(masterdf))
    return masterdf

def parse_eye_events_to_intline(line,extrainfo):
    efixspace=["","",""]
    eblinkspace=efixspace*2
    newline=line.split()
    if "EFIX" in line:
        newline.extend(efixspace)
    elif "EBLINK" in line:
        newline.extend(eblinkspace)
    newline.extend(extrainfo)
    return newline

def parse_eye_line(eye_sub, eyestring):
    """ parses each line of eye file for a given eye_phase_sub
    input one phase type list of files for a subs
    and the path to the file (in form of a string)
    outputs dataframe with all events in table
    """
    etypes=('ESACC','EFIX','EBLINK')
    study=[]
    restudy=[]
    blocks=eye_sub.block
    fnames=eye_sub.fname
    subjects=eye_sub.subject
    trialnum=0
    print(eyestring)
    studyphase = False
    for block,fname,subject in zip(blocks,fnames,subjects):
        path_file=eyestring+fname
        startcount=0
        p=Path(path_file)
        with p.open() as f:
            for line in f:
                if "studypre" in line:
                    trialnum = trialnum+1
                    studyphase = True
                elif "studypost" in line:
                    studyphase = False
                    
                if "START" in line:
                    startline=line.split()
                    starttime=int(startline[1])


                if any(e in line for e in etypes):
                    extrainfo=[starttime,trialnum,block,subject]
                    newline=parse_eye_events_to_intline(line,extrainfo)
                    if studyphase:
                        study.append(newline)
                    else:
                        restudy.append(newline)
    return study, restudy


def events_to_df(events):
    """ change raw events to data DataFrame
    then and change values to numeric"""

    eye_events_df=pd.DataFrame(events)
    eye_events_df=eye_events_df.apply(pd.to_numeric,errors='ignore')
    headers=["event","eye","start","end","duration",
    "xstart","ystart","xend","yend","?","?","trialstart",
    "trialnum","block","sub"]
    eye_events_df.columns=headers
    return eye_events_df

def eventsdf_cleanup(eye_events_df):
    x=pd.DataFrame()
    """adjust trial start time, remove irrelevant values in fixation rows,
    and then delete excess columns"""

    eyedf_clean=eye_events_df.copy()

    eyedf_clean['start']=eyedf_clean['start']-eyedf_clean['trialstart']
    eyedf_clean['end']=eyedf_clean['end']-eyedf_clean['trialstart']

    efix_mask = (eyedf_clean["event"]=="EFIX")
    eyedf_clean.loc[efix_mask, 'xend'] = np.nan
    for col in ['xstart', 'ystart', 'xend', 'yend']:
        x = eyedf_clean[col]
        eyedf_clean[col] = pd.to_numeric(x, errors='coerce')


    del eyedf_clean['trialstart']
    del eyedf_clean['?']
    del eyedf_clean['eye']

    return eyedf_clean

In [8]:
behavestring = '/Volumes/Voss_Lab/MRI/domchoicefmri_djb/behave.data/'
behavepath=Path(behavestring)
eyestring = '/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/'
eyepath=Path(eyestring)

# run functions
subids = get_subids(behavepath)

for sub in subids:
    print(sub)
    if int(sub) < 900:
        xmax = 1920/2
        ymax = 1080/2
    else:
        xmax = 1280/2
        ymax = 1024/2
        
    # process behavior
    studyfilepath = set_behavior_path(sub, behavestring, 'study')
    studyarray = read_study_file(studyfilepath)

    testfilepath = set_behavior_path(sub, behavestring, 'test')
    testarray = read_test_file(testfilepath)
    # merge study & test
    behavearray = merge_study_test(studyarray, testarray)

    # adjust coords
    behavearray = apply_adjust_pres_coords(behavearray, sub)

    # get eye files and process
    masterdf = get_eye_files(subids,eyepath)
    eye_sub = masterdf[masterdf['subject']==sub]
    study, restudy = parse_eye_line(eye_sub, eyestring)

    studydf = events_to_df(study)
    studyeyearray = eventsdf_cleanup(studydf)
    restudydf = events_to_df(restudy)
    restudeyearray = eventsdf_cleanup(restudydf)
    # concatenate study and restudy eye arrays
    studyeyearray['phase']='study'
    restudeyearray['phase']='restudy'
    eyearray = pd.concat([studyeyearray, restudeyearray], ignore_index=True)


800


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/io/parsers.py:709: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
801
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
802
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
803
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
804
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
806
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
807
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
809
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
810
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
900
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
904
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
905
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
906
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
907
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
908
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
910
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
911
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
912
/Volumes/Voss_Lab/MRI/domchoicefmri_djb/eye.data/
913
/Volumes/Voss_Lab/MRI/domcho

In [12]:
def eye_behave_combo(eyearray,behavearray):
    eyebehave=eyearray.copy()
    eyecols=eyebehave.columns.tolist()
    behavecols=['x1','y1','x2','y2','x3','y3','cond', 'dom_accuracy', 'studytrial', 'testtrial',
                'recog_accuracy', 'recog_loc_accuracy', 'cuecond']
    allcols=eyecols+behavecols
    eyebehave=eyebehave.reindex(columns=allcols)
    order_col='studytrial'
    behavearray.sort_values(by=[order_col], inplace=True)
    behavearray.set_index(order_col, drop=False, inplace=True)
    for trial in range(0,behavearray.shape[0]):
        eyetrialevents=(eyebehave['trialnum']==trial+1)
        eyetrial=eyebehave.loc[eyetrialevents]
        for col in behavecols:
            eyetrial.loc[eyetrialevents,col]=behavearray.loc[trial+1,col]

        eyebehave.loc[eyetrialevents]=eyetrial
    return eyebehave


def dist(array,x1,y1,x2,y2):
    """ distance formula for columns of coords"""
    dx=array[x1]-array[x2]
    dy=array[y1]-array[y2]
    dist=np.sqrt(dx**2+dy**2)
    return dist

def calculate_dist(eyebehave,x1,y1,name):
    """ calculate distances for start and end eye locations"""
    for x in eyebehave:
        distdict={'obj1':dist(eyebehave,x1,y1,'x1','y1'),
                        'obj2':dist(eyebehave,x1,y1,'x2','y2'),
                        'obj3':dist(eyebehave,x1,y1,'x3','y3')}

    distarray=pd.DataFrame(distdict)
    col=distarray.columns.tolist()
    distarray.columns=[c+name for c in col]
    return distarray


def loc_view(eyebehave,distarray,name):
    distarray.idxmin(axis=1)
    mindistmask=distarray.min(axis=1)<180
    distmins=distarray.loc[mindistmask]

    distminlocs=distmins.idxmin(axis=1)
    eyebehave[name]="none"
    eyebehave.loc[mindistmask,name]=distminlocs
    return eyebehave

def screenview(x,y,xmax,ymax):
    screen='screen'
    if x>xmax:
        screen='offscreen'
    if x<(0):
        screen='offscreen'
    if y>ymax:
        screen='offscreen'
    if y<(0):
        screen='offscreen'
    return screen

def assign_screenview(eyebehavedict,xname,yname,name,xmax,ymax):
    colname=name+'loc'
    for loc in eyebehavedict:
        screen=screenview(loc[xname],loc[yname],xmax,ymax)
        if loc[colname]=='none':
            loc[colname]=screen
        if name !='end':
            continue
        if loc['event']=='EFIX':
            loc[colname]=np.nan
    return eyebehavedict


def adjust_fix_before_blink(eyebehavedict):
    """replace fixations <100 ms before blinks"""
    tmp_dict=eyebehavedict.copy()
    new_previous_events=[]
    for i,ind in enumerate(tmp_dict):
        current_event = ind
        if i>0:
            if current_event['event']=='EBLINK':
                if previous_event['trialnum']==current_event['trialnum']:
                    if previous_event['event']=='EFIX' and previous_event['duration']<100:
                        previous_event['event']='blink'
            new_previous_events.append(previous_event)
        previous_event=ind
    new_previous_events.append(previous_event)
    return new_previous_events

def adjust_event_after_blink(new_previous_events):
    new_post_events=[]
    new_events=new_previous_events.copy()
    flag=False
    for current_event in new_events:
        event_type=current_event['event']
        current_trial=current_event['trialnum']
        if flag==True and previous_trial==current_trial:
            if event_type=='ESACC':
                event_type='blink'
            elif event_type=='EFIX':
                if current_event['duration']<100:
                    event_type='blink'
        new_post_events.append(current_event)
        flag=(event_type=='EBLINK')
        previous_trial=current_trial
    return new_post_events

def eyedict_backto_df(new_post_events):
    corrected_eyedf=pd.DataFrame(new_post_events)
    old_blink_mask=corrected_eyedf['event']!='EBLINK'
    corrected_eyedf=corrected_eyedf[old_blink_mask]
    corrected_eyedf.sort_values(['block','trialnum','start'])
    corrected_eyedf=corrected_eyedf.reset_index(drop=True)
    corrected_eyedf['cuecond']=corrected_eyedf['cuecond'].map({1:1, 2:0})
    corrected_eyedf['all_accuracy'] = ((corrected_eyedf['dom_accuracy']) &
                              (corrected_eyedf['recog_accuracy']))
    corrected_eyedf['dom_accuracy'] = ((corrected_eyedf['all_accuracy']) & (corrected_eyedf['cuecond']))

    return corrected_eyedf

In [15]:
# combine behavior & eye data
eyebehave = eye_behave_combo(eyearray, behavearray)

# calculate distances for start and end eye locations
startdistarray=calculate_dist(eyebehave,x1='xstart',y1='ystart',name='start')
enddistarray=calculate_dist(eyebehave,'xend','yend','end')

# start & end locations
eyebehave=loc_view(eyebehave,startdistarray,'startloc')
eyebehave=loc_view(eyebehave,enddistarray,'endloc')

#append start & end distances to eyebehave array
eyebehave=pd.concat([eyebehave, startdistarray, enddistarray], axis=1)

''' change df to dict'''
eyebehavedict=eyebehave.to_dict('records')
''' determine if non-loc viewing was on screen or offscreen'''
eyebehavedict=assign_screenview(eyebehavedict,'xstart','ystart','start',xmax, ymax)
eyebehavedict=assign_screenview(eyebehavedict,'xend','yend','end',xmax, ymax)

'''adjust artifacts in eye data due to blinks'''
new_previous_events=adjust_fix_before_blink(eyebehavedict)
corrected_eye_events=adjust_event_after_blink(new_previous_events)

'''put data back in df and remove old blinks'''
subcleandf=eyedict_backto_df(corrected_eye_events)
subcleandf['sub'] = sub

fname='data/'+sub+'eyebehave.csv'
subcleandf.to_csv(fname)
print(sub, 'is done!')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


928 is done!
